# 3) TRATAMIENTO DE VARIABLES

> Tratamento, combinación y generación de variables con valor predictivo.

- author: Iván Fernández Aguirre
- toc: true
- image: images/N3.png
- sticky_rank: 3

In [1]:
#hide
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

import numpy as np

import pandas as pd
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 50)

import warnings
warnings.filterwarnings('ignore')

In [2]:
#hide
import nbimporter
import importlib

La mayoria de las funciones utilizadas en este projecto están en el modulo:

In [3]:
import Functions_for_FMP as fffmp

In [4]:
#hide
importlib.reload(fffmp)
import Functions_for_FMP as fffmp

## CARGA DE DATOS

Empezamos, como siempre, cargando los datos. Pero, en esta oportunidad, cargaremos el *dataset* generado en el bloque 2).

In [5]:
data_clean='FMP_clean'
FMP = pd.read_pickle(data_clean)

In [6]:
fffmp.check_NaNs(FMP)

No hay missing values en el dataset :)


In [7]:
#collapse-output
display(FMP)

,id,target,home_team_name,away_team_name,match_date,league_name,league_id,is_cup,home_team_coach_id,away_team_coach_id,home_team_history_match_date_1,home_team_history_match_date_2,home_team_history_match_date_3,home_team_history_match_date_4,home_team_history_match_date_5,home_team_history_match_date_6,home_team_history_match_date_7,home_team_history_match_date_8,home_team_history_match_date_9,home_team_history_match_date_10,home_team_history_is_play_home_1,home_team_history_is_play_home_2,home_team_history_is_play_home_3,home_team_history_is_play_home_4,home_team_history_is_play_home_5,home_team_history_is_play_home_6,home_team_history_is_play_home_7,home_team_history_is_play_home_8,home_team_history_is_play_home_9,home_team_history_is_play_home_10,home_team_history_is_cup_1,home_team_history_is_cup_2,home_team_history_is_cup_3,home_team_history_is_cup_4,home_team_history_is_cup_5,home_team_history_is_cup_6,home_team_history_is_cup_7,home_team_history_is_cup_8,home_team_history_is_cup_9,home_team_history_is_cup_10,home_team_history_goal_1,home_team_history_goal_2,home_team_history_goal_3,home_team_history_goal_4,home_team_history_goal_5,home_team_history_goal_6,home_team_history_goal_7,home_team_history_goal_8,home_team_history_goal_9,home_team_history_goal_10,home_team_history_opponent_goal_1,home_team_history_opponent_goal_2,home_team_history_opponent_goal_3,home_team_history_opponent_goal_4,home_team_history_opponent_goal_5,home_team_history_opponent_goal_6,home_team_history_opponent_goal_7,home_team_history_opponent_goal_8,home_team_history_opponent_goal_9,home_team_history_opponent_goal_10,home_team_history_rating_1,home_team_history_rating_2,home_team_history_rating_3,home_team_history_rating_4,home_team_history_rating_5,home_team_history_rating_6,home_team_history_rating_7,home_team_history_rating_8,home_team_history_rating_9,home_team_history_rating_10,home_team_history_opponent_rating_1,home_team_history_opponent_rating_2,home_team_history_opponent_rating_3,home_team_history_opponent_rating_4,home_team_history_opponent_rating_5,home_team_history_opponent_rating_6,home_team_history_opponent_rating_7,home_team_history_opponent_rating_8,home_team_history_opponent_rating_9,home_team_history_opponent_rating_10,home_team_history_coach_1,home_team_history_coach_2,home_team_history_coach_3,home_team_history_coach_4,home_team_history_coach_5,home_team_history_coach_6,home_team_history_coach_7,home_team_history_coach_8,home_team_history_coach_9,home_team_history_coach_10,home_team_history_league_id_1,home_team_history_league_id_2,home_team_history_league_id_3,home_team_history_league_id_4,home_team_history_league_id_5,home_team_history_league_id_6,home_team_history_league_id_7,home_team_history_league_id_8,home_team_history_league_id_9,home_team_history_league_id_10,away_team_history_match_date_1,away_team_history_match_date_2,away_team_history_match_date_3,away_team_history_match_date_4,away_team_history_match_date_5,away_team_history_match_date_6,away_team_history_match_date_7,away_team_history_match_date_8,away_team_history_match_date_9,away_team_history_match_date_10,away_team_history_is_play_home_1,away_team_history_is_play_home_2,away_team_history_is_play_home_3,away_team_history_is_play_home_4,away_team_history_is_play_home_5,away_team_history_is_play_home_6,away_team_history_is_play_home_7,away_team_history_is_play_home_8,away_team_history_is_play_home_9,away_team_history_is_play_home_10,away_team_history_is_cup_1,away_team_history_is_cup_2,away_team_history_is_cup_3,away_team_history_is_cup_4,away_team_history_is_cup_5,away_team_history_is_cup_6,away_team_history_is_cup_7,away_team_history_is_cup_8,away_team_history_is_cup_9,away_team_history_is_cup_10,away_team_history_goal_1,away_team_history_goal_2,away_team_history_goal_3,away_team_history_goal_4,away_team_history_goal_5,away_team_history_goal_6,away_team_history_goal_7,away_team_history_goal_8,away_team_history_goal_9,away_team_history_goal_10,away_team_history_

In [8]:
#collapse-output
fffmp.Print_features(FMP)

Today Features

id
target
home_team_name
away_team_name
match_date
league_name
league_id
is_cup
home_team_coach_id
away_team_coach_id
home_team_median_rating
away_team_median_rating


Historic Features

home/away_team_history_match_date_i
home/away_team_history_is_play_home_i
home/away_team_history_is_cup_i
home/away_team_history_goal_i
home/away_team_history_opponent_goal_i
home/away_team_history_rating_i
home/away_team_history_opponent_rating_i
home/away_team_history_coach_i
home/away_team_history_league_id_i


## TRATAMIENTO, COMBINACIÓN Y GENERACION DE VARIABLES

### 1) Diferencia de *Rating*

El *rating* es una variable calculada por el proveedor de los datos (Octosport) y corresponde a la fuerza relativa calculada **antes** del partido.Por si solo, y de forma aislada, no nos da ninguna información. Sin embargo, al comparar los *ratings* de los dos equipos que juegan un partido, nos podemos dar una idea de quién es el "favorito" a ganar. Es por eso, que decidí combinar los ratings para los distintos partidos, generando una unica variable llamada "home(away)_rating_diff_i" que es la diferencia de el rating de cada equipo (*away* o *home*) contra su oponente.

Esperamos que esta variable nos de información hacerca de cuan sorprendente es un resultado respecto a lo esperado.

In [9]:
#collapse-show
for t in ['home','away']:
    for i in range(1,11):
        new_label=t+'_rating_diff_{}'.format(i)
        ranting=t+'_team_history_rating_{}'.format(i)
        op_ranting=t+'_team_history_opponent_rating_{}'.format(i)
        FMP[new_label]=FMP[ranting]-FMP[op_ranting]
        FMP=FMP.drop(columns=[ranting,op_ranting])

Se procedió de manera análoga para el partido a modelar, utilizando el rating medio calculado. Sin embargo, en este caso tuvimos que tener un cuidado especial. Sabemos que los *ratings* medios solo tienen sentido dentro de una liga. Luego, para los casos en los que los equipos pertenezcan a distintas ligas, la diferencia de *rating* la fijamos a $0$.

In [10]:
#collapse-show
for t in ['home','away']:
    C=[]
    for i in range(1,11):
        C.append(t+'_team_history_league_id_{}'.format(i))
    FMP[t+'_liga']=FMP[C].mode(axis=1)[0]
        
h='home_team_median_rating'
a='away_team_median_rating'
FMP['Rating_diff']=(FMP[h]-FMP[a])*(FMP['away_liga']==FMP['home_liga']).astype('int')*(FMP['home_liga']!=-1).astype('int')        

FMP=FMP.drop(columns=[h,a,'home_liga','away_liga'])

Finalmente, eliminamos las columnas correspondientes a *ratings*.

### 2) Diferencia de gol

De nuevo, la información más relevante para nosotros no serán los goles anotados, sinó la diferencia de goles para cada partido. Para calcular esa variable simplemente restamos los goles anotados por cada equipo (*away* o *home*) contra su oponente.

In [11]:
#collapse-show
for t in ['home','away']:
    for i in range(1,11):
        new_label=t+'_goal_diff_{}'.format(i)
        goal=t+'_team_history_goal_{}'.format(i)
        op_goal=t+'_team_history_opponent_goal_{}'.format(i)
        
        FMP[new_label] = FMP[goal]-FMP[op_goal]
        
        FMP=FMP.drop(columns=[goal,op_goal])

Finalmente, eliminamos las columnas correspondientes a goles anotados.

### 3) Continuidad del entrenador

Los cambios de entrenador muchas veces tienen un efecto animico que se traslada a los resultados. Por esta razón decidí utilizar utilizar las variables de "coach_id", para generar una variable nueva *bool* que para cada partido indique si hubo o no un cambio de entrenador. La nueva variable llamada "home(away)_coach_continuity_i" surge de comparar el identificador del entrenador en partidos consecutivos, cuando haya un cambio de entrenador valdrá $0$, y si se mantiene el mismo $1$.

(Para el partido más antiguo, $i=10$, presupondré continuidad)

In [12]:
#collapse-show
for t in ['home','away']:
    for i in range(1,10):
        
        new_label=t+'_coach_continuity_{}'.format(i)
        
        c=t+'_team_history_coach_{}'.format(i)
        a=t+'_team_history_coach_{}'.format(i+1)
        
        FMP[new_label] = ( (FMP[c]==FMP[a]) | ( (FMP[c]==-1) | (FMP[a]==-1)    ) ).astype('int')
    
    #para el partido a predecir
    new_label=t+'_coach_continuity'
    c=t+'_team_coach_id'
    a=t+'_team_history_coach_{}'.format(1)
    
    FMP[new_label] = ( (FMP[c]==FMP[a]) | ( (FMP[c]==-1) | (FMP[a]==-1)    ) ).astype('int')
    
    #para el partido más antíguo i=10
    new_label=t+'_coach_continuity_{}'.format(10)
    FMP[new_label] = np.ones(len(FMP)).astype('int')
    
    
for t in ['home','away']:
    for i in range(1,11):
        c=t+'_team_history_coach_{}'.format(i)
        FMP=FMP.drop(columns=[c])
    c=t+'_team_coach_id'
    FMP=FMP.drop(columns=[c])

Finalmente, eliminamos las columnas correspondientes al identificador de entrenador.

### 4) Acumulación de partidos

La idea aquí fue utilizar las fechas de los partidos para generar una variable que cuantifique la acumulación de partidos. Eso no solo puede tener efectos en el cansancio de los jugadores, sino también puede obligar a una mayor rotación de jugadores. De esta forma, esperamos que pueda impactar de alguna manera en el rendimiento de los equipos.

Lo que nos importará aquí, nuevamente, no será el valor absoluto, sino el diferencial entre ambos equipos. Despues de todo, si ambos estan expuestos al mismo ritmo de juegos, este factor ya pierde importancia.

Para ello, entonces, comencé por calcular el número de partidos jugados en los últimos $21$, $10$ y $4$ días, considerando los $10$ encuentros que tenemos disponibles. 

In [13]:
#collapse-show
new_label_1_i='_partidos_tres_semanas'
new_label_2_i='_partidos_diez_dias'
new_label_3_i='_partidos_cuatro_dias'

for t in ['home','away']:
    FMP[t+new_label_1_i]=np.zeros(len(FMP))
    FMP[t+new_label_2_i]=np.zeros(len(FMP))
    FMP[t+new_label_3_i]=np.zeros(len(FMP))

d0='match_date'

h='home'
a='away'

for t in ['home','away']:
    for i in range(1,11):
        di=t+'_team_history_match_date_{}'.format(i)
        FMP[t+new_label_1_i]=FMP[t+new_label_1_i]+ ((FMP[d0]-FMP[di]).dt.days<=21).astype('int')
        FMP[t+new_label_2_i]=FMP[t+new_label_2_i]+ ((FMP[d0]-FMP[di]).dt.days<=10).astype('int')
        FMP[t+new_label_3_i]=FMP[t+new_label_3_i]+ ((FMP[d0]-FMP[di]).dt.days<=4).astype('int')


Luego, generé las variables diferenciales, a través de restar los partidos acumulados para cada ventana temporal.

In [14]:
new_label_1='diff_num_partidos_tres_semanas'
new_label_2='diff_num_partidos_diez_dias'
new_label_3='diff_num_partidos_cuatro_dias'    
    
    
FMP[new_label_1]=FMP[h+new_label_1_i]-FMP[a+new_label_1_i]
FMP[new_label_2]=FMP[h+new_label_2_i]-FMP[a+new_label_2_i]
FMP[new_label_3]=FMP[h+new_label_3_i]-FMP[a+new_label_3_i]

Esperamos que valores positivos (negativos) sean perjudiciales para el equipo local (visitante).

### 5) Antigüedad de los partidos

La importancia, influencia o peso de un resultado del pasado puede depender de que tan lejos en el tiempo este. Partidos de temporadas anteriores, o incluso de antes de un corte por fecha FIFA, pueden tener menor importancia. 

Para tratar de capturar ello, se utilizó las fechas de los partidos para calcular (en días) cuanto tiempo pasó desde que se jugaron hasta el día del partido a modelar.

In [15]:
#collapse-show
d='match_date'
for t in ['home','away']:
    for i in range(1,11):
        new_label=t+'_relevance_{}'.format(i)
        c=t+'_team_history_match_date_{}'.format(i)
        
        FMP[new_label]= (FMP[d]-FMP[c]).dt.days

Una vez hecho eso, eliminé las varuables de fechas del dataset.

In [16]:
#collapse-show
d='match_date'
FMP=FMP.drop(columns=[d])
for t in ['home','away']:
    for i in range(1,11):
        c=t+'_team_history_match_date_{}'.format(i)
        FMP=FMP.drop(columns=[c])

### 6) Resultado nominal de los partidos del historial

De los partidos $i\in[1,10]$ tenemos la diferencia de gol, pero no el resultado nominal (i.e.: empate (E), victoria (V), derrota (D). Agregaremos esta variable porque nos será de utilidad a la hora de evaluar modelos o hacer verificaciones.

**Nota:** La idea no es utilizar la diferencia de goles y este resultado nominal al mismo tiempo. Esta variables estan altamente correlacionadas, por lo cual no tendría sentido. Simplemente la queremos para ver cuánto mejor es tener la diferencia de gol, o si al final con tener el resultado nominal es suficiente.

Creé esta variable directamente como un one-hot encoding. Así, tendremos las variables $D_i$ y $V_i$, que tomaran valores $0$ o $1$. Cuando ambas son 0, quedará implicito el empate.

In [17]:
#collapse-show
for t in ['home','away']:
    for i in range(1,11):
        
        new_label_V=t+'_outcome_V_{}'.format(i)
        new_label_D=t+'_outcome_D_{}'.format(i)
    
        x1=t+'_goal_diff_{}'.format(i)
        
        FMP[new_label_V]=(FMP[x1]>0).astype('int')
        FMP[new_label_D]=(FMP[x1]<0).astype('int')

### 7) Partidos amistosos

Otra cosa que puede ser interesante de tener es información sobre si un partido es o no un amistosos. Es sabido que estos partidos pueden no ser muy representativos del verdadero potencial de un equipo.

Para agregar esta variable, lo primero que hice fue identificar los "ids" de las competiciones amistosas a traves de mirar su nombre. Dado que necesitaba el nombre de las competiciones, solo usé los datos de los partidos a modelar.

In [18]:
#collapse-show
print('Ligas amistosas EN: ',FMP.league_name[FMP['league_name'].str.contains('riend')].unique())
print('Ligas amistosas ES: ',FMP.league_id[FMP['league_name'].str.contains('mist')].unique())
Friendly_competitions_ids=FMP.league_id[FMP['league_name'].str.contains('riend')].unique()

Ligas amistosas EN:  ['Club Friendlies' 'Friendly International' 'Friendly International Women'
 'Club Friendlies Women' 'Friendly International U19'
 'Friendly International U23']
Ligas amistosas ES:  []


Luego, mirando al identificador de competición para cada partico, generé la variable "is_friendly" para codificar si un partido es amistoso ($1$) o no ($0$).

In [19]:
#collapse-show
for t in ['home','away']:
    for i in range(1,11):
        league_id=t+'_team_history_league_id_{}'.format(i)
        amistoso=t+'_is_friendly_{}'.format(i)
        FMP[amistoso]=(FMP[league_id].isin(Friendly_competitions_ids)).astype('int')

In [20]:
#collapse-show
league_id='league_id'
amistoso='is_friendly'
FMP[amistoso]=(FMP[league_id].isin(Friendly_competitions_ids)).astype('int')

### 8) Resultado ponderado

Finalmente, y como un intento de mejorar el poder predictivo de mi modelo, se me ocurrió combinar las variables de diferencia de gol y diferencia de *rating*, para generar una variable que unifique el resultado de un partido con el nivel de "sorpresa" que conllevaba. A esa variable la llamé "home(away)_result_ponderado_i" y la definí como:

$Resultado\;Ponderado:=\,\,|\Delta Ratting_{std}|\,x\,\Delta Goal_{std}$

donde "$std$" refiere a que la variable esta normalizada, y $\Delta$ refiere a la diferencia.

Resultados posteriores me idicaron que no era muy útil, pero la dejo aquí como parte del trabajo.

In [21]:
#collapse-show
R=[]
G=[]
for t in ['home','away']:
    for i in range(1,11):
        g=t+'_goal_diff_{}'.format(i)
        r=t+'_rating_diff_{}'.format(i)
        G.append(g)
        R.append(r)

m_g=np.mean(FMP[G].to_numpy())
m_r=np.mean(FMP[R].to_numpy())

std_g=np.std(FMP[G].to_numpy())
std_r=np.std(FMP[R].to_numpy())

for t in ['home','away']:
    for i in range(1,11):
        new_label=t+'_result_ponderado_{}'.format(i)
        g=t+'_goal_diff_{}'.format(i)
        r=t+'_rating_diff_{}'.format(i)
        
        FMP[new_label] = np.abs((FMP[r]-m_r)/std_r)*((FMP[g]-m_g)/std_g)
        

### 10) Eliminación de features

Finalmenete eliminé del *dataset* las variables o *features* que no iba a utilizar.

In [22]:
#collapse-show
FMP=FMP.drop(columns=['home_team_name','away_team_name','league_name','league_id'])
for t in ['home','away']:
    for i in range(1,11):
        l=t+'_team_history_league_id_{}'.format(i)
        FMP=FMP.drop(columns=[l])

## VERIFICACIÓN Y GUARDADO DEL *DATASET*

Una vez terminado el proceso, verificamos el resultado y si efectivamente corregimos todos los valores faltantes.

In [23]:
#collapse-output
display(FMP)

,id,target,is_cup,home_team_history_is_play_home_1,home_team_history_is_play_home_2,home_team_history_is_play_home_3,home_team_history_is_play_home_4,home_team_history_is_play_home_5,home_team_history_is_play_home_6,home_team_history_is_play_home_7,home_team_history_is_play_home_8,home_team_history_is_play_home_9,home_team_history_is_play_home_10,home_team_history_is_cup_1,home_team_history_is_cup_2,home_team_history_is_cup_3,home_team_history_is_cup_4,home_team_history_is_cup_5,home_team_history_is_cup_6,home_team_history_is_cup_7,home_team_history_is_cup_8,home_team_history_is_cup_9,home_team_history_is_cup_10,away_team_history_is_play_home_1,away_team_history_is_play_home_2,away_team_history_is_play_home_3,away_team_history_is_play_home_4,away_team_history_is_play_home_5,away_team_history_is_play_home_6,away_team_history_is_play_home_7,away_team_history_is_play_home_8,away_team_history_is_play_home_9,away_team_history_is_play_home_10,away_team_history_is_cup_1,away_team_history_is_cup_2,away_team_history_is_cup_3,away_team_history_is_cup_4,away_team_history_is_cup_5,away_team_history_is_cup_6,away_team_history_is_cup_7,away_team_history_is_cup_8,away_team_history_is_cup_9,away_team_history_is_cup_10,home_rating_diff_1,home_rating_diff_2,home_rating_diff_3,home_rating_diff_4,home_rating_diff_5,home_rating_diff_6,home_rating_diff_7,home_rating_diff_8,home_rating_diff_9,home_rating_diff_10,away_rating_diff_1,away_rating_diff_2,away_rating_diff_3,away_rating_diff_4,away_rating_diff_5,away_rating_diff_6,away_rating_diff_7,away_rating_diff_8,away_rating_diff_9,away_rating_diff_10,Rating_diff,home_goal_diff_1,home_goal_diff_2,home_goal_diff_3,home_goal_diff_4,home_goal_diff_5,home_goal_diff_6,home_goal_diff_7,home_goal_diff_8,home_goal_diff_9,home_goal_diff_10,away_goal_diff_1,away_goal_diff_2,away_goal_diff_3,away_goal_diff_4,away_goal_diff_5,away_goal_diff_6,away_goal_diff_7,away_goal_diff_8,away_goal_diff_9,away_goal_diff_10,home_coach_continuity_1,home_coach_continuity_2,home_coach_continuity_3,home_coach_continuity_4,home_coach_continuity_5,home_coach_continuity_6,home_coach_continuity_7,home_coach_continuity_8,home_coach_continuity_9,home_coach_continuity,home_coach_continuity_10,away_coach_continuity_1,away_coach_continuity_2,away_coach_continuity_3,away_coach_continuity_4,away_coach_continuity_5,away_coach_continuity_6,away_coach_continuity_7,away_coach_continuity_8,away_coach_continuity_9,away_coach_continuity,away_coach_continuity_10,home_partidos_tres_semanas,home_partidos_diez_dias,home_partidos_cuatro_dias,away_partidos_tres_semanas,away_partidos_diez_dias,away_partidos_cuatro_dias,diff_num_partidos_tres_semanas,diff_num_partidos_diez_dias,diff_num_partidos_cuatro_dias,home_relevance_1,home_relevance_2,home_relevance_3,home_relevance_4,home_relevance_5,home_relevance_6,home_relevance_7,home_relevance_8,home_relevance_9,home_relevance_10,away_relevance_1,away_relevance_2,away_relevance_3,away_relevance_4,away_relevance_5,away_relevance_6,away_relevance_7,away_relevance_8,away_relevance_9,away_relevance_10,home_outcome_V_1,home_outcome_D_1,home_outcome_V_2,home_outcome_D_2,home_outcome_V_3,home_outcome_D_3,home_outcome_V_4,home_outcome_D_4,home_outcome_V_5,home_outcome_D_5,home_outcome_V_6,home_outcome_D_6,home_outcome_V_7,home_outcome_D_7,home_outcome_V_8,home_outcome_D_8,home_outcome_V_9,home_outcome_D_9,home_outcome_V_10,home_outcome_D_10,away_outcome_V_1,away_outcome_D_1,away_outcome_V_2,away_outcome_D_2,away_outcome_V_3,away_outcome_D_3,away_outcome_V_4,away_outcome_D_4,away_outcome_V_5,away_outcome_D_5,away_outcome_V_6,away_outcome_D_6,away_outcome_V_7,away_outcome_D_7,away_outcome_V_8,away_outcome_D_8,away_outcome_V_9,away_outcome_D_9,away_outcome_V_10,away_outcome_D_10,home_is_friendly_1,home_is_friendly_2,home_is_friendly_3,home_is_friendly_4,home_is_friendly_5,home_is_friendly_6,home_is_friendly_7,home_is_friendly_8,home_is_friendly_9,home_is_friendly_10,away_is_friendly_1,away_is_friendly_2,away_is_friendly_3

In [24]:
fffmp.check_NaNs(FMP)

No hay missing values en el dataset :)


In [25]:
#collapse-output
FMP.describe()

,id,is_cup,home_team_history_is_play_home_1,home_team_history_is_play_home_2,home_team_history_is_play_home_3,home_team_history_is_play_home_4,home_team_history_is_play_home_5,home_team_history_is_play_home_6,home_team_history_is_play_home_7,home_team_history_is_play_home_8,home_team_history_is_play_home_9,home_team_history_is_play_home_10,home_team_history_is_cup_1,home_team_history_is_cup_2,home_team_history_is_cup_3,home_team_history_is_cup_4,home_team_history_is_cup_5,home_team_history_is_cup_6,home_team_history_is_cup_7,home_team_history_is_cup_8,home_team_history_is_cup_9,home_team_history_is_cup_10,away_team_history_is_play_home_1,away_team_history_is_play_home_2,away_team_history_is_play_home_3,away_team_history_is_play_home_4,away_team_history_is_play_home_5,away_team_history_is_play_home_6,away_team_history_is_play_home_7,away_team_history_is_play_home_8,away_team_history_is_play_home_9,away_team_history_is_play_home_10,away_team_history_is_cup_1,away_team_history_is_cup_2,away_team_history_is_cup_3,away_team_history_is_cup_4,away_team_history_is_cup_5,away_team_history_is_cup_6,away_team_history_is_cup_7,away_team_history_is_cup_8,away_team_history_is_cup_9,away_team_history_is_cup_10,home_rating_diff_1,home_rating_diff_2,home_rating_diff_3,home_rating_diff_4,home_rating_diff_5,home_rating_diff_6,home_rating_diff_7,home_rating_diff_8,home_rating_diff_9,home_rating_diff_10,away_rating_diff_1,away_rating_diff_2,away_rating_diff_3,away_rating_diff_4,away_rating_diff_5,away_rating_diff_6,away_rating_diff_7,away_rating_diff_8,away_rating_diff_9,away_rating_diff_10,Rating_diff,home_goal_diff_1,home_goal_diff_2,home_goal_diff_3,home_goal_diff_4,home_goal_diff_5,home_goal_diff_6,home_goal_diff_7,home_goal_diff_8,home_goal_diff_9,home_goal_diff_10,away_goal_diff_1,away_goal_diff_2,away_goal_diff_3,away_goal_diff_4,away_goal_diff_5,away_goal_diff_6,away_goal_diff_7,away_goal_diff_8,away_goal_diff_9,away_goal_diff_10,home_coach_continuity_1,home_coach_continuity_2,home_coach_continuity_3,home_coach_continuity_4,home_coach_continuity_5,home_coach_continuity_6,home_coach_continuity_7,home_coach_continuity_8,home_coach_continuity_9,home_coach_continuity,home_coach_continuity_10,away_coach_continuity_1,away_coach_continuity_2,away_coach_continuity_3,away_coach_continuity_4,away_coach_continuity_5,away_coach_continuity_6,away_coach_continuity_7,away_coach_continuity_8,away_coach_continuity_9,away_coach_continuity,away_coach_continuity_10,home_partidos_tres_semanas,home_partidos_diez_dias,home_partidos_cuatro_dias,away_partidos_tres_semanas,away_partidos_diez_dias,away_partidos_cuatro_dias,diff_num_partidos_tres_semanas,diff_num_partidos_diez_dias,diff_num_partidos_cuatro_dias,home_relevance_1,home_relevance_2,home_relevance_3,home_relevance_4,home_relevance_5,home_relevance_6,home_relevance_7,home_relevance_8,home_relevance_9,home_relevance_10,away_relevance_1,away_relevance_2,away_relevance_3,away_relevance_4,away_relevance_5,away_relevance_6,away_relevance_7,away_relevance_8,away_relevance_9,away_relevance_10,home_outcome_V_1,home_outcome_D_1,home_outcome_V_2,home_outcome_D_2,home_outcome_V_3,home_outcome_D_3,home_outcome_V_4,home_outcome_D_4,home_outcome_V_5,home_outcome_D_5,home_outcome_V_6,home_outcome_D_6,home_outcome_V_7,home_outcome_D_7,home_outcome_V_8,home_outcome_D_8,home_outcome_V_9,home_outcome_D_9,home_outcome_V_10,home_outcome_D_10,away_outcome_V_1,away_outcome_D_1,away_outcome_V_2,away_outcome_D_2,away_outcome_V_3,away_outcome_D_3,away_outcome_V_4,away_outcome_D_4,away_outcome_V_5,away_outcome_D_5,away_outcome_V_6,away_outcome_D_6,away_outcome_V_7,away_outcome_D_7,away_outcome_V_8,away_outcome_D_8,away_outcome_V_9,away_outcome_D_9,away_outcome_V_10,away_outcome_D_10,home_is_friendly_1,home_is_friendly_2,home_is_friendly_3,home_is_friendly_4,home_is_friendly_5,home_is_friendly_6,home_is_friendly_7,home_is_friendly_8,home_is_friendly_9,home_is_friendly_10,away_is_friendly_1,away_is_friendly_2,away_is_friendly_3,away_i

In [26]:
#collapse-output
fffmp.Print_features(FMP)

Today Features

id
target
is_cup
Rating_diff
home_coach_continuity
away_coach_continuity
home_partidos_tres_semanas
home_partidos_diez_dias
home_partidos_cuatro_dias
away_partidos_tres_semanas
away_partidos_diez_dias
away_partidos_cuatro_dias
diff_num_partidos_tres_semanas
diff_num_partidos_diez_dias
diff_num_partidos_cuatro_dias
is_friendly


Historic Features

home/away_team_history_is_play_home_i
home/away_team_history_is_cup_i
home/away_rating_diff_i
home/away_goal_diff_i
home/away_coach_continuity_i
home/away_relevance_i
home/away_outcome_V_i
home/away_outcome_D_i
home/away_is_friendly_i
home/away_result_ponderado_i


Y, finalmente, procedemos a guardar el *dataset* resultante para poder cargarlo en otros notebooks.

In [27]:
data_clean='FMP_final'
FMP.to_pickle(data_clean)